## Introduction
In this example, we are building a GlassFlow pipeline to encrypt email addresses from event data in real time. 

## Pre-requisite

- Create your free GlassFlow account via the [GlassFlow WebApp](https://app.glassflow.dev).
- Get your Personal Access Token to authorize the Python SDK to interact with GlassFlow Cloud.
- Run `pip install -r requirements.txt`to install the necessary Python dependencies for this example.

## Build Pipeline

In [ ]:
import glassflow

In [ ]:
# method to create an AES_KEY
def create_key():
    import secrets
    key = secrets.token_bytes(16)
    return key.hex()

Please replace this variable with your own Personal Access Token from https://app.glassflow.dev/profile

In [ ]:
glassflow_personal_access_token = "< your-personal-access-token >"

Obtain the GlassFlow client and use the personal_access_token to authorize the client to connect with GlassFlow Cloud.

In [ ]:
client = glassflow.GlassFlowClient(personal_access_token=glassflow_personal_access_token)

Use one of the existing GlassFlow spaces to deploy the pipeline.

In [ ]:
spaces = client.list_spaces()
space = spaces.spaces[0]

In [ ]:
print(space['name'])
print(space['id'])

Display the `transform.py` file, which contains the Python code that we submit to the pipeline.

In [21]:
%pycat transform.py

from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
import base64
import os


# Function to encrypt email address
def handler(data, log):
    key = os.getenv("AES_KEY").encode('utf-8')

    email = data['email']

    # AES key must be either 16, 24, or 32 bytes
    cipher = AES.new(key, AES.MODE_CBC)  # CBC mode with a random IV
    iv = cipher.iv  # Get the Initialization Vector (IV)

    # Pad the email to be a multiple of 16 bytes
    padded_email = pad(email.encode(), AES.block_size)

    # Encrypt the padded email
    encrypted_email = cipher.encrypt(padded_email)

    # Return IV + encrypted email encoded in base64 (to make it easier to store or send)
    encrypted_email = base64.b64encode(iv + encrypted_email).decode()
    data['email_encrypted'] = encrypted_email
    return data


Create a AES_KEY and add it as an environment variable to the pipeline

In [ ]:
aes_key = create_key()

In [ ]:
env_vars = [{
  "name": "AES_KEY",
  "value": aes_key
}]

The transform code depends on a python library. Add a requirements.txt to the pipeline

In [ ]:
with open("requirements.txt", 'r') as file:
    requirements_txt = file.read()
print(requirements_txt)

In [ ]:
# Create the pipeline with transform.py file and name it `email-encryption` in the main space
pipeline = client.create_pipeline(
    name="email-encryption", 
    transformation_file='transform.py',
    space_id=space['id'], 
    env_vars=env_vars, 
    requirements=requirements_txt
)

## Publish events to the pipeline

In [ ]:
# Create a random data generator to generate some dummy events data
from faker import Faker
def random_datagen():
    fake = Faker()
    return {
        "name": fake.name(),
        "email": fake.email(),
        "id": fake.uuid4()
    }

In [ ]:
data_source = pipeline.get_source()

In [ ]:
# Generate some data and send it to the pipeline. Store it locally to compare
input_data = []
for i in range(10):
    d = random_datagen()
    input_data.append(d)
    r = data_source.publish(d)

In [ ]:
display(input_data)

## Consume events from the pipeline 

Obtain the sink to consume the transformed events from the pipeline.

In [ ]:
data_sink = pipeline.get_sink()

In [ ]:
output_data = []
for i in range(10):
    resp = data_sink.consume()
    output_data.append(resp.json())

In [ ]:
display(output_data)

In [ ]:
## Explore the pipeline on the web-UI
pipeline_url = f"https://app.glassflow.dev/pipelines/{pipeline.id}"
print(pipeline_url)